In [6]:
import numpy as np
import os
import threading
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, GaussianNoise, MaxPooling1D, UpSampling1D
from src.preprocess.dim_reduce.patient_split import *

In [7]:
%cd /Users/Kunal/Projects/TCH_CardiacSignals_F20/
filepath = "Working_Data/Training_Subset/Normalized/two_hbs/Normalized_Fixed_Dim_HBs_Idx1.npy"
data = np.load(filepath)
ratio = 0.3
training, test, full = patient_split_all(filepath, ratio)

/Users/kunal/Projects/TCH_CardiacSignals_F20


In [5]:
data.shape

(23937, 200, 4)

In [8]:
TIME_STEPS=30

def create_sequences(X, time_steps=TIME_STEPS):
    Xs= []
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)

    return np.array(Xs)

X_train, y_train = create_sequences(training, training)
X_test, y_test = create_sequences(test, test)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(X_train.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(X_train.shape[2])))
model.compile(optimizer='adam', loss='mse')
model.summary()



In [10]:
latent_dim = 100
# Build the encoder
encoder_inputs = keras.Input(shape=(1000, 4))
encoder = Sequential()
encoder.add(InputLayer((1000,4)))
encoder.add(Conv1D(10, 11, activation="linear", padding="same"))
encoder.add(Conv1D(10, 7, activation="relu", padding="same"))
encoder.add(MaxPooling1D(2))
encoder.add(Flatten())
encoder.add(Dense(750, activation = 'tanh', kernel_initializer='glorot_normal'))
encoder.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
encoder.add(Dense(400, activation = 'relu', kernel_initializer='glorot_normal'))
encoder.add(Dense(300, activation='relu', kernel_initializer='glorot_normal'))
encoder.add(Dense(200, activation = 'relu', kernel_initializer='glorot_normal'))
encoder.add(Dense(latent_dim))
# encoder.add(MaxPooling1D(2, padding="same"))

# encoder.add(Conv1D(64, 3, activation="relu", padding="same"))
# encoder.add(MaxPooling1D(1, padding="same"))
#
# encoder.add(BatchNormalization())
#
# encoder.add(Conv1D(128, 3, activation="relu", padding="same"))
# encoder.add(Conv1D(64, 3, activation="relu", padding="same"))
# encoder.add(MaxPooling1D(5, padding="same"))
#
# encoder.add(BatchNormalization())
# encoder.add(Conv1D(32, 3, activation="relu", padding="same"))
# encoder.add(Conv1D(16, 3, activation="relu", padding="same"))
# encoder.add(Conv1D(1, 3, activation="relu", padding="same"))

encoder.summary()
####################################################################################################################
# Build the decoder

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 1000, 10)          450       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 10)          710       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 10)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 750)               3750750   
_________________________________________________________________
dense_7 (Dense)              (None, 500)               375500    
_________________________________________________________________
dense_8 (Dense)              (None, 400)              

In [11]:
decoder = Sequential()
decoder.add(InputLayer((latent_dim,)))
decoder.add(Dense(200, activation = 'tanh', kernel_initializer='glorot_normal'))
decoder.add(Dense(300, activation='relu', kernel_initializer='glorot_normal'))
decoder.add(Dense(400, activation = 'relu', kernel_initializer='glorot_normal'))
decoder.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
decoder.add(Dense(750, activation = 'relu', kernel_initializer='glorot_normal'))
decoder.add(Dense(10000, activation = 'relu', kernel_initializer='glorot_normal'))
decoder.add(Reshape((1000,10)))
decoder.add(Conv1DTranspose(8, 7, activation="relu", padding="same"))
decoder.add(Conv1DTranspose(4, 11, activation="linear", padding="same"))



decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 200)               20200     
_________________________________________________________________
dense_13 (Dense)             (None, 300)               60300     
_________________________________________________________________
dense_14 (Dense)             (None, 400)               120400    
_________________________________________________________________
dense_15 (Dense)             (None, 500)               200500    
_________________________________________________________________
dense_16 (Dense)             (None, 750)               375750    
_________________________________________________________________
dense_17 (Dense)             (None, 10000)             7510000   
_________________________________________________________________
reshape (Reshape)            (None, 1000, 10)         

In [26]:
def build_autoencoder(sig_shape, encode_size):
    """
    Builds a deterministic autoencoder, returning both the encoder and decoder models
    :param sig_shape: shape of input signal
    :param encode_size: dimension that we want to reduce to
    :return: encoder, decoder models
    """
    latent_dim = 100
    # Build the encoder
    encoder = Sequential()
    encoder.add(InputLayer((1000,4)))
    encoder.add(Conv1D(10, 7, activation="linear", padding="same"))
    encoder.add(Flatten())
    encoder.add(Dense(750, activation = 'tanh', kernel_initializer='glorot_normal'))
    encoder.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
    encoder.add(Dense(400, activation = 'relu', kernel_initializer='glorot_normal'))
    encoder.add(Dense(300, activation='relu', kernel_initializer='glorot_normal'))
    encoder.add(Dense(200, activation = 'relu', kernel_initializer='glorot_normal'))
    encoder.add(Dense(latent_dim))
    # encoder.summary()
    ####################################################################################################################
    # Build the decoder

    decoder = Sequential()
    decoder.add(InputLayer((latent_dim,)))
    decoder.add(Dense(200, activation = 'tanh', kernel_initializer='glorot_normal'))
    decoder.add(Dense(300, activation='relu', kernel_initializer='glorot_normal'))
    decoder.add(Dense(400, activation = 'relu', kernel_initializer='glorot_normal'))
    decoder.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
    decoder.add(Dense(750, activation = 'relu', kernel_initializer='glorot_normal'))
    decoder.add(Dense(10000, activation = 'relu', kernel_initializer='glorot_normal'))
    decoder.add(Reshape((1000,10)))
    decoder.add(Conv1DTranspose(4, 7, activation="relu", padding="same"))

    return encoder, decoder

In [53]:
def read_in(file_index, normalized):
    """
    Reads in a file and can toggle between normalized and original files
    :param file_index: patient number as string
    :param normalized: boolean that determines whether the files should be normalized or not
    :return: returns npy array of patient data across 4 leads
    """
    if normalized == 1:
        data = np.load(os.path.join("Working_Data", "Normalized_Fixed_Dim_HBs_Idx" + file_index + ".npy"))
    else:
        data = np.load(os.path.join("Working_Data", "Fixed_Dim_HBs_Idx" + file_index + ".npy"))

    return data

In [19]:
def training_ae(num_epochs, reduced_dim, file_index):
    """
    Training function for deterministic autoencoder, saves the encoded and reconstructed arrays
    :param num_epochs: number of epochs to use
    :param reduced_dim: goal dimension
    :param file_index: patient number
    :return: None
    """
    data = read_in(file_index,1)
    signal_shape = data.shape[1:]
    encoder, decoder = build_autoencoder(signal_shape, reduced_dim)

    inp = Input(signal_shape)
    encode = encoder(inp)
    reconstruction = decoder(encode)

    autoencoder = Model(inp, reconstruction)
    autoencoder.compile(optimizer='adam', loss='mse')

    mod = autoencoder.fit(x=data, y=data, epochs=num_epochs)

    encoded = encoder.predict(data)
    reconstruction = decoder.predict(encoded)

    reconstruction_save = os.path.join("Working_Data", "reconstructed_ae_" + str(reduced_dim) + "d_Idx" + str(file_index) + ".npy")
    encoded_save = os.path.join("Working_Data", "reduced_ae_" + str(reduced_dim) + "d_Idx" + str(file_index) + ".npy")
    np.save(reconstruction_save, reconstruction)
    np.save(encoded_save,encoded)


def run_over(num_epochs, encoded_dim):
    """
    Run training autoencoder over all dims in list
    :param dims: dimension to run on
    :return None, saves arrays for reconstructed and dim reduced arrays
    """
    indices = ['1']

    # ,'4','5','6','7','8','10','11','12','14','16','17','18','19','20','21','22','25','27','28','30','31','32',
    #             '33','34','35','37','38','39','40','41','42','44','45','46','47','48','49','50','52','53','54','55','56'

    for patient_ in indices:
        print("Starting on index: " + str(patient_))
        training_ae(num_epochs, encoded_dim, patient_)
        print("Completed " + patient_ + " reconstruction and encoding")


In [28]:
from src.preprocess.dim_reduce.patient_split import *
train, test, full = patient_split_all(os.path.join("Working_Data", "1000d", "Normalized_Fixed_Dim_HBs_Idx35.npy"), 0.3)

In [29]:
encoder, decoder = build_autoencoder(1000, 100)
inp = Input(train.shape[1:])
encode = encoder(inp)
reconstruction = decoder(encode)
autoencoder = Model(inp, reconstruction)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x=train, y=train, epochs=100)

Epoch 1/100
36/36 [==============================] - 7s 187ms/step - loss: 0.6411